# Sample Data Understanding
The goal of this exercise is to explore the source data sets - `PhenotypeData.csv` and `CollectionData.csv` - and understand the schema, data types and relationships. This will inform the data model design

In [1]:
import pandas as pd
import os

source_collections = os.path.join(os.getcwd(),'source_data\CollectionsData.csv')
source_phenotype = os.path.join(os.getcwd(),'source_data\PhenotypeData.csv')

## Collections Data

In [2]:
df_collections = pd.read_csv(source_collections)
df_collections.head()

,sampid,HID,"Isolate # (blank = ""1"")",Date Collected
0,dzd-1,6662880545010918781,NaN,7/30/18
1,dzd-2,6662880545010918781,NaN,7/30/18
2,dzd-3,7662883545019918757,NaN,7/30/18
3,dzd-4,219934676530263734,NaN,7/30/18
4,dzd-5,219928676527263683,NaN,7/30/18


In [5]:
df_collections.describe(include='all')

,sampid,HID,"Isolate # (blank = ""1"")",Date Collected
count,1648,1.648000e+03,299.000000,1648
unique,1648,NaN,NaN,12
top,dzd-1,NaN,NaN,10/7/18
freq,1,NaN,NaN,288
mean,NaN,4.492027e+18,2.545151,NaN
std,NaN,2.710123e+18,1.020167,NaN
min,NaN,1.652753e+16,2.000000,NaN
25%,NaN,2.105663e+18,2.000000,NaN
50%,NaN,4.320262e+18,2.000000,NaN
75%,NaN,6.883161e+18,3.000000,NaN


In [7]:
df_collections.rename({'Isolate # (blank = "1")': 'isolate', 'Date Collected': 'date_collected'}, axis='columns', inplace=True)

In [9]:
df_collections.head()

,sampid,HID,"Isolate # (blank = ""1"")",date_collected
0,dzd-1,6662880545010918781,NaN,7/30/18
1,dzd-2,6662880545010918781,NaN,7/30/18
2,dzd-3,7662883545019918757,NaN,7/30/18
3,dzd-4,219934676530263734,NaN,7/30/18
4,dzd-5,219928676527263683,NaN,7/30/18


In [10]:
df_collections.columns = ['sampid', 'hid', 'isolate', 'date_collected']
df_collections.head(1)

,sampid,hid,isolate,date_collected
0,dzd-1,6662880545010918781,NaN,7/30/18


In [11]:
df_collections['isolate'] = df_collections.isolate.fillna(1)
df_collections.head(2)

,sampid,hid,isolate,date_collected
0,dzd-1,6662880545010918781,1.0,7/30/18
1,dzd-2,6662880545010918781,1.0,7/30/18


### So far...
1. dataset contains 1648 rows. `sampid` is unique across the dataset
2. `HID` and `Date Collected` do not have any `NULL`s
3. Filled blanks in `isolate` with `1`

### to determine
1. Is the combination of `hid+isolate` unique?
2. What date range does the data correspond to?

In [14]:
df_temp = df_collections[['hid', 'isolate']].value_counts().reset_index(name='count')
df_temp.head()

,hid,isolate,count
0,3662881545109918953,1.0,3
1,1904146365886763004,1.0,3
2,2105669412858572953,1.0,2
3,8662889545020918780,1.0,2
4,2457907026153730647,1.0,2


In [17]:
len(df_temp[df_temp['count'] > 1])

19

In [21]:
pd.to_datetime(df_collections['date_collected'])

0      2018-07-30
1      2018-07-30
2      2018-07-30
3      2018-07-30
4      2018-07-30
          ...    
1643   2018-11-27
1644   2018-11-27
1645   2018-11-27
1646   2018-11-27
1647   2018-11-27
Name: date_collected, Length: 1648, dtype: datetime64[ns]

In [24]:
df_collections['date_collected'] = pd.to_datetime(df_collections['date_collected'])
df_collections['date_collected'].describe()

C:\Users\bmare\AppData\Local\Temp\ipykernel_23224\2192369267.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_collections['date_collected'].describe()


count                    1648
unique                     12
top       2018-10-07 00:00:00
freq                      288
first     2018-07-30 00:00:00
last      2018-11-27 00:00:00
Name: date_collected, dtype: object

### Conclusion 
* Data is collected between 2018 July 30th and Novermber 27th
* There are 19 combinations of `hid + isolate` that map to more than one `sampid`
    * 1608 out of the 1648 are uniquely identifiable using `hid + isolate`


## Phenotype Data

In [25]:
df_pheno = pd.read_csv(source_phenotype)
df_pheno.head(3)

,HID,Isolate,Received,Organism,Source,Test,Antibiotic,Value,Antibiotic Interpretation,Method
0,5513475385954145565,ISO2,07/01/2018 0129,Citrobacter freundii,Wound,Susceptibility,Tobramycin,8,Intermediate,VITEK II
1,5513475385954145565,ISO3,07/01/2018 0129,Proteus vulgaris/penneri,Wound,Susceptibility,Nitrofurantoin,128,Resistant,VITEK II; MYLA
2,5513475385954145565,ISO2,07/01/2018 0129,Citrobacter freundii,Wound,Susceptibility,Cefazolin,<=4,Resistant,VITEK II


In [26]:
df_pheno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56372 entries, 0 to 56371
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   HID                        56372 non-null  int64 
 1   Isolate                    56372 non-null  object
 2   Received                   56372 non-null  object
 3   Organism                   56372 non-null  object
 4   Source                     54864 non-null  object
 5   Test                       56372 non-null  object
 6   Antibiotic                 56372 non-null  object
 7   Value                      55834 non-null  object
 8   Antibiotic Interpretation  52601 non-null  object
 9   Method                     56372 non-null  object
dtypes: int64(1), object(9)
memory usage: 4.3+ MB


In [27]:
df_temp = df_pheno[['HID', 'Isolate']].value_counts().reset_index(name='count')
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3808 entries, 0 to 3807
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   HID      3808 non-null   int64 
 1   Isolate  3808 non-null   object
 2   count    3808 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 89.4+ KB


In [28]:
df_temp = df_pheno[['HID', 'Isolate', 'Antibiotic']].value_counts().reset_index(name='count')
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56008 entries, 0 to 56007
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   HID         56008 non-null  int64 
 1   Isolate     56008 non-null  object
 2   Antibiotic  56008 non-null  object
 3   count       56008 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.7+ MB


In [31]:
df_pheno['Received'] = pd.to_datetime(df_pheno['Received'])
df_pheno['Received'].describe(datetime_is_numeric=True)

count                            56372
mean     2018-08-26 18:12:47.826935040
min                2018-07-01 01:29:00
25%                2018-07-31 11:03:00
50%                2018-08-24 15:32:00
75%                2018-09-24 08:32:00
max                2018-10-23 20:53:00
Name: Received, dtype: object

In [35]:
len(df_pheno['Antibiotic'].unique())

88

In [43]:
sorted(df_pheno['Antibiotic'].unique())

['5-Fluorocytosine',
 'AMOXICILLIN + CLAVULANATE',
 'AZTREONAM',
 'Amikacin',
 'Amox/Clavulanic Acid',
 'Amp/Sulbactam',
 'Amphotericin B',
 'Ampicillin',
 'Ampicillin/Sulbactam',
 'Anidulafungin',
 'Aztreonam',
 'CEFAZOLIN',
 'CEFOXITIN',
 'CEFTAZIDIME/AVIBACTAM',
 'CEFTOLOZANE/TAZOBACTAM',
 'COLISTIN',
 'Caspofungin',
 'Cefazolin',
 'Cefepime',
 'Cefotaxime',
 'Cefotaxime w/Clavulanic Acid',
 'Cefoxitin Screen',
 'Ceftaroline',
 'Ceftazidime',
 'Ceftazidime w/Clavulanic Acid',
 'Ceftazidime/avibactam',
 'Ceftolozane/tazobactam',
 'Ceftriaxone',
 'Cefuroxime',
 'Chloramphenicol',
 'Ciprofloxacin',
 'Clarithromycin',
 'Clindamycin',
 'Colistin',
 'DOXYCYCLINE',
 'Daptomycin',
 'ESBL',
 'ESBL - Cefotaxime',
 'ESBL - Cefotaxime/Clavu',
 'ESBL - Ceftazidime/Clavu',
 'ESBL Confirmation:',
 'Ertapenem',
 'Erythromycin',
 'FOSFOMYCIN',
 'Fluconazole',
 'Fosfomycin',
 'Gentamicin',
 'Gentamicin Synergy',
 'Imipenem',
 'Inducible Clindamycin',
 'Inducible Clindamycin Resistance',
 'Isavuconazo

In [45]:
df_temp = df_pheno['Organism'].unique()
print(len(df_temp), sorted(df_temp))

124 ['Achromobacter dentrificans', 'Achromobacter species', 'Achromobacter xylosoxidans', 'Acinetobacter baumannii', 'Acinetobacter baumannii complex', 'Acinetobacter haemolyticus', 'Acinetobacter lwoffii', 'Acinetobacter radioresistens', 'Acinetobacter ursingii', 'Aerococcus urinae', 'Alpha-hemolytic strep', 'Bacillus cereus group', 'Bordetella', 'Brevundimonas diminuta', 'Burkholderia cepacia', 'Burkholderia cepacia group', 'Burkholderia gladioli', 'Burkholderia species', 'Candida albicans', 'Candida dubliniensis', 'Candida glabrata', 'Candida guilliermondii', 'Candida krusei', 'Candida lusitaniae', 'Candida parapsilosis', 'Candida tropicalis', 'Chryseobacterium indologenes', 'Citrobacter amalonaticus', 'Citrobacter braakii', 'Citrobacter freundii', 'Citrobacter koseri', 'Citrobacter species', 'Citrobacter youngae', 'Coagulase negative Staphylococcus', 'Coagulase positive Staphylococcus', 'Corynebacterium species', 'Curvularia species', 'Edwardsiella tarda', 'Enterobacter aerogenes',

In [46]:
df_temp = df_pheno['Method'].unique()
print(len(df_temp), sorted(df_temp))

9 [' KB METHOD', ' KB METHOD; MANUAL METHOD', ' MANUAL METHOD', ' MICROSCAN', ' MYLA', ' PHENO ACCELERATE 1', ' PHENO ACCELERATE 1; MANUAL METHOD', ' VITEK II', ' VITEK II; MYLA']


### Conclusion
* There are 56372 observations in the Phenotype Data
* `Source`, `Value` and `Antibiotic Interpretation` fields may contains `NULL`s
* Even when `Value` is present, `Antibiotic Interpretation` may be `NULL`
* Data corresponds to 3008 unique samples, representing 124 organisms tested across 88 antibiotics
* Data is observed in 2018 between July 1st and October 23rd